In [1]:
cd ../

/Users/linafaik/Documents/projects/time-series-forecasting-models


In [2]:
import pandas as pd
import numpy as np
import os

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS, NBEATS
from neuralforecast.losses.pytorch import MQLoss

from config import *
from src.data_processing import *
from src.metrics import *
from src.training import *
from src.viz import *

%load_ext autoreload
%autoreload 2

/Users/linafaik/Documents/projects/time-series-forecasting-models/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-21 10:37:46,490	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-05-21 10:37:46,695	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
force = False

## Data loading

In [4]:
df = pd.read_csv(path_data_processed)
df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
df.head()

,date,store_id,state_id,sold_quantity,sold_amount,event_type_1,event_type_2,event_sporting,event_cultural,event_national,event_religious
0,2011-01-29,CA_1,CA,4337,10933.16,NaN,NaN,0,0,0,0
1,2011-01-29,CA_2,CA,3494,9101.52,NaN,NaN,0,0,0,0
2,2011-01-29,CA_3,CA,4739,11679.83,NaN,NaN,0,0,0,0
3,2011-01-29,CA_4,CA,1625,4561.59,NaN,NaN,0,0,0,0
4,2011-01-29,TX_1,TX,2556,6586.68,NaN,NaN,0,0,0,0


## Model training

In [5]:
name_scenario = "dl_models"

In [6]:
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'
os.environ['PYTORCH_MPS_FORCE_DISABLE'] = '1' 

In [7]:
# Define the maximum number of training steps for the deep learning models
max_steps = 500

# Set the path where the forecast results will be saved (based on the scenario name)
path = os.path.join("output", name_scenario, f"forecasts_{name_scenario}.csv")

# If recomputation is forced or the file doesn't exist yet
if force or not os.path.exists(path):

    # Split the dataset into training and test sets based on forecasting horizon
    train_df, test_df = split_train_test(
        df=df, 
        horizon=H,                  # Forecast horizon
        column_date=time_col,      # Name of the date column
        column_id=id_col,          # Name of the time series identifier column
    )

    # Display the number of rows in each split
    print(f"{len(train_df)} rows for train")
    print(f"{len(test_df)} rows for test")
    
    # List of dynamic features (external events) to include as covariates
    columns_dynamic_feat = [
        'event_sporting', 'event_cultural', 
        'event_national', 'event_religious'
    ]

    # Define input window size (e.g., 4 times the forecast horizon)
    input_size = 4 * H
    
    # Define the forecasting models to train with probabilistic loss (90% quantile)
    models = [
        NHITS(
            input_size=input_size, 
            h=H, 
            max_steps=max_steps,
            loss=MQLoss(level=[90])     # Quantile loss for prediction intervals
        ),
        NBEATS(
            input_size=input_size, 
            h=H, 
            max_steps=max_steps,
            loss=MQLoss(level=[90])     # Same loss function as NHITS
        ),
    ]

    # Instantiate the NeuralForecast object with the models and frequency
    nf = NeuralForecast(models=models, freq=freq)
    
    # Fit the models to the training data (including external dynamic features)
    nf.fit(
        df=train_df[[id_col, time_col, target_col] + columns_dynamic_feat],
        id_col=id_col,
        time_col=time_col,
        target_col=target_col
    )
    
    # Generate predictions for the test horizon with confidence intervals
    forecasts_df = nf.predict(level=[90])
    
    # Merge the predictions with the test set to evaluate performance later
    forecasts_enr_df = (
        test_df
        .merge(forecasts_df, on=[id_col, time_col], how="left")
    )
    
    # Concatenate the train and enriched test set into one final DataFrame
    forecasts_enr_df = pd.concat([train_df, forecasts_enr_df], axis=0).reset_index(drop=True)

    # Create the output directory if it doesn't already exist
    os.makedirs(os.path.join("output", name_scenario), exist_ok=True)

    # Save the final dataset with forecasts to disk
    forecasts_enr_df.to_csv(path, index=False)

else:
    # If forecasts already exist, load them from the saved CSV
    forecasts_enr_df = pd.read_csv(path)

# Display the last few rows of the final forecast dataset
forecasts_enr_df.tail()

,date,store_id,state_id,sold_quantity,sold_amount,event_type_1,event_type_2,event_sporting,event_cultural,event_national,event_religious,NHITS-median,NHITS-lo-90,NHITS-hi-90,NBEATS-median,NBEATS-lo-90,NBEATS-hi-90
19405,2016-05-18,WI_3,WI,3268,9163.29,NaN,NaN,0,0,0,0,9388.312,8049.8022,10411.233,8962.190,7848.9297,9986.906
19406,2016-05-19,WI_3,WI,3398,9660.13,NaN,NaN,0,0,0,0,9280.735,8048.7760,10383.619,9247.131,8227.1980,10229.908
19407,2016-05-20,WI_3,WI,4126,11982.37,NaN,NaN,0,0,0,0,10612.061,9182.0670,11701.540,10976.120,9879.1000,12282.996
19408,2016-05-21,WI_3,WI,4519,12370.23,NaN,NaN,0,0,0,0,12475.693,11032.9760,14058.324,12602.628,11297.1780,14294.606
19409,2016-05-22,WI_3,WI,4757,13432.85,NaN,NaN,0,0,0,0,12222.669,10595.5100,13889.405,12591.581,10886.4370,14096.614


## Results analysis

In [8]:
# Initialize containers:
# - `scores` will store overall evaluation metrics per model
# - `scores_per_ts` will store evaluation metrics for each individual time series (by ID)
scores = {}
scores_per_ts = []

# Loop through each model’s median forecast column
for column in ['NHITS-median', 'NBEATS-median']:
    
    # Filter out rows with missing forecast values for the current model
    forecasts_filtered_df = forecasts_enr_df[forecasts_enr_df[column].notna()]
    
    # Compute overall evaluation metrics (e.g., MAE, RMSE) between ground truth and forecast
    scores[column] = evaluate(
        forecasts_filtered_df[target_col], 
        forecasts_filtered_df[column]
    )
    
    # Compute metrics for each time series individually (grouped by id_col)
    scores_per_ts_model_df = (
        forecasts_filtered_df
        .groupby(id_col)
        .apply(lambda group: pd.Series(
            evaluate(group[target_col], group[column])
        ))
        .reset_index()
    )
    
    # Add a column to tag each row with the model name
    scores_per_ts_model_df["model"] = column
    
    # Keep only model name and metrics columns, and append to the global list
    scores_per_ts.append(
        scores_per_ts_model_df[["model"] + [c for c in scores_per_ts_model_df.columns if c != "model"]]
    )

# Convert the dictionary of global scores into a DataFrame
scores_df = (
    pd.DataFrame(scores).T.reset_index()
    .rename(columns={"index": "model"})
)

# Concatenate all time-series-level score DataFrames into one
scores_per_ts_df = pd.concat(scores_per_ts, axis=0).reset_index(drop=True)

/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_3104/3694950192.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: pd.Series(
/var/folders/x1/5lcr60ps53nfbsvw0s9dt0640000gn/T/ipykernel_3104/3694950192.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda group: pd.Series(


In [9]:
scores_df

,model,MAE,RMSE,MAPE,R2,count
0,NHITS-median,1163.569738,2.296756e+06,0.077986,0.865390,140.0
1,NBEATS-median,1058.553978,1.987458e+06,0.071381,0.883518,140.0


In [10]:
list_metrics = ["MAPE"]
plot_global_scores(scores_df=scores_df, list_metrics=list_metrics)

In [11]:
plot_scores_per_ts(scores_per_ts_df, column_id = id_col, list_metrics=list_metrics)

In [13]:
rnd_id = np.random.choice(scores_per_ts_df[id_col].unique())

for column in ['NHITS-median', 'NBEATS-median']:
    
    print(f"Model: {column}") 

    plot_forecast_with_ci(
        forecasts_enr_df, 
        column_id=id_col,
        column_date=time_col,
        column_target=target_col,
        column_forecast=column,
        model_name=column.split("-")[0],
        uid=rnd_id, 
        train_tail=30,
        ).show()

Model: NHITS-median


/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:183: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:184: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:185: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



Model: NBEATS-median


/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:183: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:184: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/Users/linafaik/Documents/projects/time-series-forecasting-models/src/viz.py:185: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

